In [36]:
# unzip input file first

import xml.etree.ElementTree as ET
import requests
import math

In [2]:
root = ET.parse('input-files/RoughlyTravisCounty.osm').getroot()

In [8]:
latitudes = []
longitudes = []
for type_tag in root.findall("node"):
    d = type_tag.attrib
    latitudes.append(float(d["lat"]))
    longitudes.append(float(d["lon"]))

In [50]:
LATITUDE_MILES = 69
LONGITUDE_MILES = 54.6

vertical_distance = (max(latitudes) - min(latitudes))*LATITUDE_MILES
horizontal_distance = (max(longitudes) - min(longitudes))*LONGITUDE_MILES

In [51]:
# Assuming we want to make 400 squares, how should we partition the vertically/horizontally
NUM_REGIONS = 400
square_len = (vertical_distance*horizontal_distance/NUM_REGIONS)**0.5

In [52]:
latitude_delta = square_len/LATITUDE_MILES
longitude_delta = square_len/LONGITUDE_MILES
longitude_delta

0.039167692270157484

In [53]:
lat_lon = zip(latitudes, longitudes)

In [62]:
min_lat = min(latitudes)
min_lon = min(longitudes)
max_lat = max(latitudes)
max_lon = max(longitudes)
nrows = math.ceil((max_lat - min_lat) / latitude_delta)
ncols = math.ceil((max_lon - min_lon) / longitude_delta)
def map_point_to_region(latitude, longitude):
    return math.floor((latitude-min_lat)/latitude_delta) * ncols  + math.floor((longitude-min_lon)/longitude_delta)

In [74]:
def get_representative(region_num): # 8 cols per row, # R10 --> row_num = 1, col_num = 2
    row_num = region_num//ncols # Correct
    col_num = region_num - row_num*ncols
    lat = min_lat + row_num * latitude_delta + 0.5*latitude_delta
    lon = min_lon + col_num * longitude_delta + 0.5*longitude_delta
    return [lat, lon]

In [70]:
map_point_to_region(30.084446, -97.702885)

13

In [71]:
get_representative(13)

(30.090783782593846, -97.68468325435288)

In [67]:
for i in range(400):
    get_representative(i)

0.030993565187689837

In [72]:
 headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624884518d9d4075408e95e3401165976247',
    'Content-Type': 'application/json; charset=utf-8'
}

In [76]:
body = {"locations": [get_representative(i) for i in range(400)]}

In [78]:
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

In [80]:
call._content # 400*400 = 160000

b'{"error":{"code":6004,"message":"Request parameters exceed the server configuration limits. Only a total of 3500 routes are allowed."},"info":{"engine":{"version":"6.3.0","build_date":"2020-09-21T01:00:26Z"},"timestamp":1600714613837}}'

In [84]:
160000/

200.0

In [82]:
3500**0.5

59.16079783099616